In [5]:
!pip install crewai crewai_tools dotenv

/opt/homebrew/anaconda3/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=35503) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


  Using cached dotenv-0.9.9-py2.py3-none-any.whl.metadata (279 bytes)
Using cached dotenv-0.9.9-py2.py3-none-any.whl (1.9 kB)


In [6]:
pip install langchain langchain-google-genai python-dotenv


  Using cached langchain_google_genai-2.1.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.17-py3-none-any.whl.metadata (9.8 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
Using cached langchain_google_genai-2.1.2-py3-none-any.whl (42 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached google_ai_generativelanguage-0.6.17-py3-none-any.whl (1.4 MB)
Using cached google_api_core-2.24.2-py3-none-any.whl (160 kB)
Using cached proto_plus-1.26.1-py3-none-any.whl (50 kB)
Note: you may need to restart the kernel to use updated packages.


### Google Gemini API (Pro)

In [ ]:
from dotenv import load_dotenv
import os
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from .env
load_dotenv()

# Get your API key
KEY = os.getenv("GOOGLE_API_KEY")

# Initialize the Gemini 1.5 Pro LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",  # 
    verbose=True,
    temperature=0.5,
    google_api_key=KEY
)


In [173]:
response = llm.invoke("Say something cool about artificial intelligence.")
print(response)


content="AI isn't just about robots and algorithms; it's about augmenting human potential.  Imagine a future where AI helps us unlock scientific breakthroughs we can't even fathom today, from curing diseases to understanding the universe.  That's the cool part – it's a tool that can help us achieve the extraordinary." additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []} id='run-bd64f289-9a09-4b0d-a9ee-bc1259a24ad0-0' usage_metadata={'input_tokens': 7, 'output_tokens': 68, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}}


### tools to be used

In [ ]:

class DatamuseTool:
    name = "Datamuse Tool"
    description = "Semantic keyword suggestion from Datamuse API"

    def _run(self, topic: str):
        import requests
        response = requests.get(f"https://api.datamuse.com/words?ml={topic}&max=5")
        return [res["word"] for res in response.json()]

In [99]:
tool = DatamuseTool()
keywords = tool._run("artificial intelligence")
print(keywords)


['ai', 'artificial general intelligence', 'machine learning', 'automated machine learning', 'intelligent system']


In [118]:
class NewsTool:
    name = "NewsData Tool"
    description = "Fetch trending news for a topic from NewsData.io"

    def _run(self, topic: str):
        API_KEY = os.getenv("NEWSDATA_API_KEY")
        if not API_KEY:
            return "API key not found. Please set the NEWSDATA_API_KEY environment variable."

        url = "https://newsdata.io/api/1/latest"
        params = {
            "apikey": API_KEY,
            "q": topic,
            "language": "en"
        }

        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data.get("status") != "success":
                return f"API error: {data.get('message', 'Unknown error')}"

            articles = data.get("results", [])
            if not articles:
                return "No recent news found."

            return "\n".join([f"{a['title']} - {a['link']}" for a in articles[:3]])

        except requests.exceptions.RequestException as e:
            return f"Request failed: {e}"


In [105]:
tool = NewsTool()
news = tool._run("artificial intelligence")
print(news)


3 Incredible Growth Stocks That Are Now Too Cheap to Ignore - https://www.fool.com/investing/2025/04/15/3-incredible-growth-stocks-that-are-now-too-cheap/
'Marrying Technology With Love For Cricket': Joy Bhattacharjya On Latest Fantasy Offering - https://www.outlookindia.com/sports/cricket/joy-bhattacharjya-fifs-sports-data-gameathon-explained-ai-technology-fantasy-gaming
Global tariff conflicts prompt calls for Korea-U.S. sector collaboration - https://koreajoongangdaily.joins.com/news/2025-04-15/business/industry/Global-tariff-conflicts-prompt-calls-for-KoreaUS-sector-collaboration/2285674


In [ ]:
import requests

class QuotableTool:
    name = "Quotable Tool"
    description = "Fetch quotes from Quotable.io by keyword"

    def _run(self, topic: str):
        try:
            url = f"https://api.quotable.io/quotes?query={topic}&limit=3"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                quotes = data.get("results", [])
                if quotes:
                    return "\n".join([f'"{q.get("content")}" — {q.get("author")}' for q in quotes])
                else:
                    return f"No quotes found for topic: '{topic}'"
            else:
                return f"API error (status code {response.status_code})"

        except requests.exceptions.RequestException as e:
            return f"Network error while accessing Quotable.io:\n{e}"








In [164]:
tool = QuotableTool()
print(tool._run("technology"))




⚠️ Network error while accessing Quotable.io:
HTTPSConnectionPool(host='api.quotable.io', port=443): Max retries exceeded with url: /quotes?query=technology&limit=3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x300287230>: Failed to resolve 'api.quotable.io' ([Errno 8] nodename nor servname provided, or not known)"))


In [165]:
print(tool._run("success"))

⚠️ Network error while accessing Quotable.io:
HTTPSConnectionPool(host='api.quotable.io', port=443): Max retries exceeded with url: /quotes?query=success&limit=3 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x300286090>: Failed to resolve 'api.quotable.io' ([Errno 8] nodename nor servname provided, or not known)"))


In [ ]:
import os
import requests
from dotenv import load_dotenv

class MultiSourceQuoteTool:
    name = "MultiSourceQuoteTool"
    description = "Fetch quotes from both FavQs and ZenQuotes APIs"

    def __init__(self):
        load_dotenv()
        self.favqs_key = os.getenv("FAVQS_API_KEY")
        if not self.favqs_key:
            raise ValueError("Missing FAVQS_API_KEY in .env")

    def fetch_favqs_quotes(self, topic: str):
        headers = {"Authorization": f"Token token={self.favqs_key}"}
        params = {"filter": topic, "type": "tag"}

        try:
            response = requests.get("https://favqs.com/api/quotes", headers=headers, params=params, timeout=10)
            if response.status_code != 200:
                return []

            quotes = response.json().get("quotes", [])
            result = []

            for q in quotes:
                body = q.get("body", "").strip()
                author = q.get("author", "Unknown").strip() if q.get("author") else "Unknown"
                if body:
                    result.append(f'"{body}" — {author}')

            return result[:3]

        except requests.exceptions.RequestException:
            return []

    def fetch_zenquotes(self, topic: str):
        try:
            response = requests.get("https://zenquotes.io/api/quotes", timeout=10)
            if response.status_code != 200:
                return []

            quotes = response.json()
            matches = []
            for q in quotes:
                quote_text = q.get("q", "")
                if topic.lower() in quote_text.lower():
                    matches.append(f'"{quote_text.strip()}" — {q.get("a", "Unknown").strip()}')
                if len(matches) >= 3:
                    break
            return matches

        except requests.exceptions.RequestException:
            return []

    def _run(self, topic: str):
        favqs = self.fetch_favqs_quotes(topic)
        zen = self.fetch_zenquotes(topic)

        combined = list(dict.fromkeys(favqs + zen))  # Deduplicate

        if not combined:
            return f" No quotes found for topic: '{topic}'"

        return "\n\n".join(combined[:5])



In [171]:
tool = MultiSourceQuoteTool()
print(tool._run("technology"))


"As technology advances, it reverses the characteristics of every situation again and again. The age of automation is going to be the age of 'do it yourself.'" — Marshall McLuhan

"You'd think that in this age, especially in the 21st century - especially with all the technology and all the discoveries that we've made - that we would figure out how to tackle abuse." — Tori Amos

"I hate to mention age, but I come from an era when we weren't consumed by technology and television." — Jimmy Buffett


In [172]:
tool = MultiSourceQuoteTool()
print(tool._run("success"))


"Only work half a day. It doesn't matter which half you work -- the first 12 hours or the second 12 hours." — Kemmons Wilson

"I do not think there is any other quality so essential to success, of any kind, as perseverance. It overcomes almost everything." — John D. Rockefeller

"People really don’t like to hear success explained away as luck — especially successful people." — Michael Lewis

"Success is good at any age, but the sooner you find it, the longer you will enjoy it." — Napoleon Hill

"Success is a poor teacher." — Robert Kiyosaki


### Defining AGENTS

In [200]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Optional
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import datetime

    


In [201]:
# Define State Schema
class BlogState(TypedDict):
    topic: str
    tone: Optional[str]
    subtopics: Optional[list]
    research_data: Optional[dict]
    draft: Optional[str]
    seo_data: Optional[dict]
    exports: Optional[dict]
    error: Optional[str]

In [202]:
# Initialize Tools
quote_tool = MultiSourceQuoteTool()
news_tool = NewsTool()
keyword_tool = DatamuseTool()

## Defining the nodes 

### planner node

In [273]:
async def planner_node(state: Dict[str, Any]):
    if not state.get('topic'):
        return {"error": "No topic provided"}

    prompt = f"""
    Break down the following blog topic into exactly 3 subtopics and suggest a recommended depth level.

    Topic: {state['topic']}
    Tone: {state.get('tone', 'neutral')}

    Return in this exact format:
    Subtopic1|Subtopic2|Subtopic3
    Recommended depth: basic

    Do not explain anything. Only return the 2-line format.
    """

    try:
        response = await llm.ainvoke(prompt)
        raw_output = response.content.strip()
        print("🔍 LLM Response from planner_node:\n", raw_output)

        parts = raw_output.split("\n")

        if len(parts) < 2 or "|" not in parts[0] or "Recommended depth:" not in parts[1]:
            return {"error": f"Unexpected format from LLM:\n{raw_output}"}

        subtopics = parts[0].split("|")
        depth = parts[1].split(":")[-1].strip()

        return {
            "subtopics": subtopics,
            "depth_level": depth
        }

    except Exception as e:
        return {"error": f"Planning failed: {str(e)}"}
 


In [274]:
# Fake blog state
test_state = {
    "topic": "How Python is used in AI",
    "tone": "educational"
}


In [275]:
import asyncio

result = await planner_node(test_state)
print("Planner Node Result:")
print(result)



🔍 LLM Response from planner_node:
 Libraries and Frameworks for AI in Python|Machine Learning with Python|Deep Learning with Python
Recommended depth: basic
Planner Node Result:
{'subtopics': ['Libraries and Frameworks for AI in Python', 'Machine Learning with Python', 'Deep Learning with Python'], 'depth_level': 'basic'}


### research node

In [277]:
import json

async def research_node(state: BlogState):
    # Run all tool outputs first
    quote_result = quote_tool._run(state['topic'])
    news_result = news_tool._run(state['topic'])
    keyword_result = keyword_tool._run(state['topic'])

    # If no quote found, generate one using LLM
    if "no quote" in quote_result.lower():
        quote_prompt = f'Write a one-line original inspirational quote about: "{state["topic"]}"'
        try:
            quote_response = await llm.ainvoke(quote_prompt)
            quote_result = f'"{quote_response.content.strip()}" — AI'
        except Exception as e:
            quote_result = f'"[Quote generation failed: {e}]"'

    research = {
        "quotes": quote_result,
        "news": news_result,
        "keywords": keyword_result
    }

    return {"research_data": research}


In [219]:
## testing the node

test_state = {
    "topic": "How Python is used in AI",
    "tone": "educational",
    "subtopics": [
        "Popular AI Libraries",
        "Python for Data Preprocessing",
        "Real-world Projects in AI"
    ]
}


In [220]:
research_result = await research_node(test_state)
print(" Research Output:")
print(research_result)


 Research Output:
{'research_data': {'quotes': '"Python empowers AI, weaving complex algorithms into intelligent tapestries of code." — AI', 'news': 'GPT-4.1 Free On Windsurf AI Coding Assistant for 7 Days - https://www.geeky-gadgets.com/gpt-4-1-free-on-windsurf/\nThe Rise of Cloud Experts: What You Should Know in 2025 - https://www.dailyexcelsior.com/the-rise-of-cloud-experts-what-you-should-know-in-2025/\nTop 10 Generative AI Courses in India in 2025 - https://www.analyticsinsight.net/artificial-intelligence/top-10-generative-ai-courses-in-india-in-2025', 'keywords': ['metarule', 'did', 'was', 'anaconda', 'artificial insemination']}}


### writing node 

In [278]:
async def writer_node(state: BlogState):
    prompt = f"""
    Write blog post with:
    Topic: {state['topic']}
    Tone: {state['tone']}
    Subtopics: {state['subtopics']}
    Research: {json.dumps(state['research_data'])}
    
    Include Markdown formatting with H2 headings.
    """
    
    try:
        response = await llm.ainvoke(prompt)
        return {"draft": response.content}
    except Exception as e:
        return {"error": f"Writing failed: {str(e)}"}

In [221]:
test_state = {
    "topic": "How Python is used in AI",
    "tone": "educational",
    "subtopics": [
        "Popular AI Libraries",
        "Data Preprocessing with Python",
        "Real-world AI Projects"
    ],
    "research_data": {
        "quotes": '"Python is the soul of AI development." — AI',
        "news": "OpenAI releases a new version of GPT model with better fine-tuning capabilities.",
        "keywords": ["Python", "AI", "TensorFlow", "Machine Learning"]
    }
}


In [225]:
# Run the writer node
result = await writer_node(test_state)

# Extract content
blog_draft = result.get("draft") or result.get("error")

# Print to console
print("log Draft Output:")
print(blog_draft)




log Draft Output:
## How Python Fuels the AI Revolution

Python has become the dominant language in the world of Artificial Intelligence (AI), and for good reason. Its versatility, extensive libraries, and supportive community make it the perfect tool for developing cutting-edge AI applications.  As someone aptly put it, "\"Python is the soul of AI development.\" — AI".  This post will explore how Python contributes to the exciting field of AI, covering key areas like essential libraries, data preprocessing techniques, and real-world applications.

## Popular AI Libraries

Python's strength in AI stems largely from its rich ecosystem of dedicated libraries. These libraries provide pre-built functions and tools that simplify complex tasks, allowing developers to focus on the core logic of their AI projects.  Here are a few of the most popular:

* **TensorFlow:** Developed by Google, TensorFlow is a powerful library for numerical computation and large-scale machine learning. It's particu

In [226]:
with open("blog_draft_test_output.txt", "w", encoding="utf-8") as f:
    f.write(blog_draft)

print(" Blog draft saved to blog_draft_test_output.txt")

 Blog draft saved to blog_draft_test_output.txt


### seo node

In [279]:
async def seo_node(state: BlogState):
    prompt = f"""
    Generate SEO metadata for the blog content below.

    Content:
    {state['draft']}

    Keywords: {state['research_data']['keywords']}

    ONLY return a valid JSON object in this format (no explanations, no markdown):

    {{
        "title": "",
        "meta_description": "",
        "keywords": [],
        "reading_time": "",
        "slug": ""
    }}
    """

    try:
        response = await llm.ainvoke(prompt)

        # Clean code block if wrapped
        raw = response.content.strip()
        if raw.startswith("```"):
            raw = raw.strip("```json").strip("```")

        return {"seo_data": json.loads(raw)}
    except Exception as e:
        return {"error": f"SEO failed: {str(e)}"}


In [231]:
with open("/Users/vijay/Documents/PROJECTS ML/blog agent.ai/blog_draft_test_output.txt", "r", encoding="utf-8") as f:
    draft_text = f.read()


In [232]:
test_state = {
    "draft": draft_text,
    "research_data": {
        "keywords": ["Python", "AI", "Machine Learning", "TensorFlow", "Automation"]
    }
}


In [233]:
seo_result = await seo_node(test_state)

# Display result
print(" SEO Metadata:")
print(seo_result.get("seo_data") or seo_result.get("error"))


 SEO Metadata:
{'title': 'How Python Fuels the AI Revolution', 'meta_description': "Discover how Python's versatility, libraries like TensorFlow and PyTorch, and data preprocessing tools are driving the advancement of Artificial Intelligence. Explore real-world AI applications and the future of AI with Python.", 'keywords': ['Python', 'AI', 'Machine Learning', 'TensorFlow', 'PyTorch', 'Scikit-learn', 'Keras', 'Data Preprocessing', 'Pandas', 'NumPy', 'Image Recognition', 'Natural Language Processing', 'Self-Driving Cars', 'Medical Diagnosis', 'Fraud Detection', 'Automation'], 'reading_time': '3-5 minutes', 'slug': 'how-python-fuels-the-ai-revolution'}


In [234]:
import json

# Run the SEO node
seo_result = await seo_node(test_state)

# Print result
print("🔍 SEO Metadata:")
print(seo_result.get("seo_data") or seo_result.get("error"))

# Save to JSON file if SEO was successful
seo_data = seo_result.get("seo_data")
if seo_data:
    output_path = "/Users/vijay/Documents/PROJECTS ML/blog agent.ai/seo_metadata.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(seo_data, f, indent=2)
    print(f"SEO metadata saved to {output_path}")
else:
    print(" SEO metadata not saved due to error.")


🔍 SEO Metadata:
{'title': 'How Python Fuels the AI Revolution', 'meta_description': "Discover how Python's versatility, libraries like TensorFlow and PyTorch, and data preprocessing tools are driving the advancement of Artificial Intelligence. Explore real-world AI applications and the future of Python in AI.", 'keywords': ['Python', 'AI', 'Machine Learning', 'TensorFlow', 'PyTorch', 'Scikit-learn', 'Keras', 'Data Preprocessing', 'Pandas', 'NumPy', 'Image Recognition', 'Natural Language Processing', 'Self-Driving Cars', 'Medical Diagnosis', 'Fraud Detection', 'Automation'], 'reading_time': '3-5 minutes', 'slug': 'how-python-fuels-the-ai-revolution'}
SEO metadata saved to /Users/vijay/Documents/PROJECTS ML/blog agent.ai/seo_metadata.json


### export and summarize agent

In [286]:
async def execution_agent(state: Dict[str, Any]) -> Dict[str, Any]:
    try:
        if not state.get('seo_data'):
            return {"error": "SEO data missing"}
        if not state.get('draft'):
            return {"error": "Blog content missing"}

        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        slug = state['seo_data'].get('slug', 'blog-post')
        filename_base = f"{slug}-{timestamp}"

        md_content = f"# {state['seo_data']['title']}\n\n{state['draft']}"
        md_filename = f"{filename_base}.md"
        with open(md_filename, 'w', encoding='utf-8') as f:
            f.write(md_content)

        metadata = {
            "seo": state['seo_data'],
            "system": {
                "generated_at": timestamp,
                "word_count": len(state['draft'].split()),
                "source_tools": ["NewsData.io", "Datamuse API", "Quotable.io"]
            }
        }
        json_filename = f"{filename_base}-metadata.json"
        with open(json_filename, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2)

        summary = f"""
🚀 Blog Generation Complete
{'-'*40}
📝 Title: {state['seo_data']['title']}
📊 Word Count: {len(state['draft'].split())}
📂 Files Created:
  - {os.path.abspath(md_filename)}
  - {os.path.abspath(json_filename)}
🔑 Keywords: {', '.join(state['seo_data']['keywords'][:5])}
⏱️ Generation Time: {timestamp}
"""

        return {
            "exports": {
                "markdown": md_filename,
                "metadata": json_filename
            },
            "cli_summary": summary,
            "validation": {
                "markdown_exists": os.path.exists(md_filename),
                "metadata_valid": bool(state['seo_data'])
            }
        }

    except Exception as e:
        return {"error": f"Export failed: {str(e)}"}

In [236]:
import json

# Load blog draft
with open("/Users/vijay/Documents/PROJECTS ML/blog agent.ai/blog_draft_test_output.txt", "r", encoding="utf-8") as f:
    draft = f.read()

# Load SEO metadata
with open("/Users/vijay/Documents/PROJECTS ML/blog agent.ai/seo_metadata.json", "r", encoding="utf-8") as f:
    seo_data = json.load(f)

# Construct state
test_state = {
    "draft": draft,
    "seo_data": seo_data
}


In [237]:
result = await execution_agent(test_state)

# Print summary or error
if "error" in result:
    print(" Error:", result["error"])
else:
    print(result["cli_summary"])



        🚀 Blog Generation Complete
        ----------------------------------------
        📝 Title: How Python Fuels the AI Revolution
        📊 Word Count: 587
        📂 Files Created:
          - /Users/vijay/Documents/PROJECTS ML/blog agent.ai/how-python-fuels-the-ai-revolution-20250416-034919.md
          - /Users/vijay/Documents/PROJECTS ML/blog agent.ai/how-python-fuels-the-ai-revolution-20250416-034919-metadata.json
        🔑 Keywords: Python, AI, Machine Learning, TensorFlow, PyTorch
        ⏱️ Generation Time: 20250416-034919
        


### build the graph

In [287]:
# Build and Connect Graph
workflow = StateGraph(BlogState)

# Add Nodes
workflow.add_node("planner", planner_node)
workflow.add_node("researcher", research_node)
workflow.add_node("writer", writer_node)
workflow.add_node("seo_optimizer", seo_node)
workflow.add_node("executor", execution_agent)

# Set Up Conditional Flow
workflow.set_entry_point("planner")

In [288]:
# Planner -> Researcher
workflow.add_conditional_edges(
    "planner",
    lambda state: "error" if state.get("error") else "researcher",
    {"error": END, "researcher": "researcher"}
)

# Researcher -> Writer
workflow.add_conditional_edges(
    "researcher",
    lambda state: "error" if state.get("error") else "writer",
    {"error": END, "writer": "writer"}
)

# Writer -> SEO Optimizer
workflow.add_conditional_edges(
    "writer",
    lambda state: "error" if state.get("error") else "seo_optimizer",
    {"error": END, "seo_optimizer": "seo_optimizer"}
)

# SEO Optimizer -> Executor
workflow.add_conditional_edges(
    "seo_optimizer",
    lambda state: "error" if state.get("error") else "executor",
    {"error": END, "executor": "executor"}
)


In [289]:
# Final Step
workflow.add_edge("executor", END)

# Compile Workflow
blog_chain = workflow.compile()

In [ ]:
# Prompt user
topic = input("Enter blog topic: ").strip()
tone = input("Enter content tone (professional/casual/technical): ").strip()

# ✅ Initial state for LangGraph
initial_state = {
    "topic": topic,
    "tone": tone
}

#  Reuse execution agent
async def run_export_summary(state):
    try:
        result = await execution_agent(state)

        if "cli_summary" in result:
            print(result["cli_summary"])
        else:
            print("❌ Failed to generate summary:", result.get("error"))

    except Exception as e:
        print(f"❌ Automation failed: {e}")

# ✅ Run the full pipeline 
async def test_blog_graph():
    result = await blog_chain.ainvoke(initial_state)

    if "error" in result:
        print("❌ Error during blog generation:", result["error"])
    elif "draft" in result and "seo_data" in result:
        print("✅ Blog draft and metadata generated. Exporting...")
        await run_export_summary(result)
    else:
        print("⚠️ Blog generated but missing required fields. Full result:\n")
        print(result)

# ✅ Run the test
await test_blog_graph()




🔍 LLM Response from planner_node:
 Solar Power Advancements|Wind Energy Innovations|Energy Storage Technologies
Recommended depth: advanced
✅ Blog draft and metadata generated. Exporting...

🚀 Blog Generation Complete
----------------------------------------
📝 Title: Renewable Energy Trends: Catching the Winds of Change
📊 Word Count: 543
📂 Files Created:
  - /Users/vijay/Documents/PROJECTS ML/blog agent.ai/renewable-energy-trends-20250416-043159.md
  - /Users/vijay/Documents/PROJECTS ML/blog agent.ai/renewable-energy-trends-20250416-043159-metadata.json
🔑 Keywords: renewable energy, solar energy, wind energy, energy storage, solar power
⏱️ Generation Time: 20250416-043159

